In [1]:
#-------------------------- arguments ---------------------------#
options(stringsAsFactors=F)
library(glmnet)
library(foreach)
library(genio)
library(Rcpp)

 

Loading required package: Matrix

Loading required package: foreach

Loaded glmnet 2.0-13




In [3]:
list.files()

[1] "adjust_expr_updated.R"                                      
 [2] "adjust_expr.R"                                              
 [3] "adjust_expression_gtex8_debug.R"                            
 [4] "adjust_expression_gtex8_parallel.R"                         
 [5] "adjust_expression_gtex8.R"                                  
 [6] "check_adjusted_expression.ipynb"                            
 [7] "cmds_gen.R"                                                 
 [8] "correlation_snp_exp_mapping_MetaXcan.ipynb"                 
 [9] "correlation_snp_exp.ipynb"                                  
[10] "ctimp_working_version-Copy1.ipynb"                          
[11] "ctimp_working_version.ipynb"                                
[12] "EDA.ipynb"                                                  
[13] "expression_adjustment_miRNA.ipynb"                          
[14] "expression_adjustment_output.ipynb"                         
[15] "expression_adjustment.ipynb"                                
[16] "expression_check.ipynb"                                     
[17] "extract_dosage_info.ipynb"                                  
[18] "extract_raw_expression_v0.R"                                
[19] "extract_raw_expression-Copy1.R"                             
[20] "extract_raw_expression.R"                                   
[21] "gene_id_prep.ipynb"                                         
[22] "gene_info.ipynb"                                            
[23] "gtex_v7_Adipose_Subcutaneous_imputed_eur_covariances.txt.gz"
[24] "predictdb_overall.ipynb"                                    
[25] "predictdb_summary-Copy1.ipynb"                              
[26] "predictdb_summary.ipynb"                                    
[27] "predictdb_weight_mapping.ipynb"                             
[28] "predictdb-Copy2.ipynb"                                      
[29] "predictdb.ipynb"                                            
[30] "snp_dosage_info-Copy1.ipynb"                                
[31] "snp_dosage_info.ipynb"                                      
[32] "summarize_snp_direction.R"                                  
[33] "test_c_function.ipynb"                                      
[34] "test_ctimp_faster-Copy1.ipynb"                              
[35] "test_ctimp_faster-Copy2.ipynb"                              
[36] "test_ctimp_faster.ipynb"                                    
[37] "test_ctimp_singleest.ipynb"                                 
[38] "test_ctimp_xdimension-Copy1.ipynb"                          
[39] "test_ctimp_xdimension.ipynb"                                
[40] "test_ctimp-Copy1.ipynb"                                     
[41] "test_ctimp-Copy1.ipynb.invalid"                             
[42] "test_ctimp-Copy2.ipynb"                                     
[43] "test_ctimp-Copy3.ipynb"                                     
[44] "test_ctimp.ipynb"                                           
[45] "Untitled.ipynb"

In [ ]:
options(stringsAsFactors=F)
args = commandArgs(trailingOnly=TRUE)
chr = as.numeric(args[1])
info_list <- list()
genotype_dir <- "/ysm-gpfs/project/wl382/GTEx_v8/genotype/cis_loc/"


count = 0

cat(paste0("INFO: chr ", chr))
info_list[[chr]] <- list()
gtex_dir <- "/gpfs/loomis/project/zhao/zy92/GTEX/" 
chr_str <- paste0("chr", chr, "/")
#gene_vec <- list.files(paste0(genotype_dir, chr_str))
gene_vec = dir(paste0(gtex_dir, "adjusted_expr/chr", chr))

for (gene_id in gene_vec) {
    count = count + 1
    if (count %% 50 == 0) {cat(paste0("INFO: chr", chr, " gene ", count))}
    dose_path <- paste0(genotype_dir, chr_str, gene_id, "/", gene_id)
    Yt <- dir(paste0(gtex_dir, "adjusted_expr/", chr_str, gene_id, "/"))
    P <- length(Yt)
    Y = list()

    for(t in 1:P){
        Y[[t]] = read.table(paste0(gtex_dir, "adjusted_expr/chr", chr, "/", gene_id, "/", Yt[t]), header=F)
    }

    ssize = unlist(lapply(Y, nrow))
    T_num = length(Yt)
    info_list[[chr]][[gene_id]] <- list(T_num, ssize, Y)
    }

save(info_list, 
     file = paste0("/gpfs/loomis/project/zhao/zy92/GTEX/gene_info/info_list_chr", chr, ".RData"))


In [9]:
#-------------------------- data import ---------------------------#  

info_list <- list()
genotype_dir <- "/ysm-gpfs/project/wl382/GTEx_v8/genotype/cis_loc/"

count = 0
for (chr in 1:22) {
    IRdisplay::display_html(paste0("INFO: chr ", chr))
    info_list[[chr]] <- list()
    gtex_dir <- "/gpfs/loomis/project/zhao/zy92/GTEX/" 
    chr_str <- paste0("chr", chr, "/")
    #gene_vec <- list.files(paste0(genotype_dir, chr_str))
    gene_vec = dir(paste0(gtex_dir, "adjusted_expr/chr", chr))
    
    for (gene_id in gene_vec) {
        count = count + 1
        if (count %% 50 == 0) {IRdisplay::display_html(paste0("INFO: chr", chr, " gene ", count))}
        dose_path <- paste0(genotype_dir, chr_str, gene_id, "/", gene_id)
        Yt <- dir(paste0(gtex_dir, "adjusted_expr/", chr_str, gene_id, "/"))
        P <- length(Yt)
        Y = list()

        for(t in 1:P){
            Y[[t]] = read.table(paste0(gtex_dir, "adjusted_expr/chr", chr, "/", gene_id, "/", Yt[t]), header=F)
        }
        
        ssize = unlist(lapply(Y, nrow))
        T_num = length(Yt)
        info_list[[chr]][[gene_id]] <- list(T_num, ssize, Y)
        }

}



INFO: chr 1

INFO: chr1 gene 50

INFO: chr1 gene 100

INFO: chr1 gene 150

INFO: chr1 gene 200

...

INFO: chr22 gene 25650

INFO: chr22 gene 25700

INFO: chr22 gene 25750

INFO: chr22 gene 25800

INFO: chr22 gene 25850

In [ ]:
getwd()

In [2]:




#-------------------------- functions ---------------------------#  

grad_prep <- function(X, Y){
	## pre-calculate some metrics for gradient
	ll = length(Y)
	P = ncol(X[[1]])
	XY = matrix(0,P,ll)
	for(i in 1:ll){
		XY[,i] = t(X[[i]])%*%Y[[i]]/nrow(X[[i]])
	}
	XY
}

cv_helper <- function(N, fold){
	valid_num = floor(N/fold)
	set.seed(123)
	perm = sample(1:N, size = N)
	idx1 = seq(1,N,valid_num)
	idx2 = c(idx1[-1]-1,N)
	list(perm=perm, idx=cbind(idx1,idx2))
}

minmax_lambda <- function(lst){
	max_lam = max(unlist(lapply(lst, function(x){max(x$lambda)})))
	min_lam = min(unlist(lapply(lst, function(x){min(x$lambda)})))
	c(min_lam, max_lam)
}

elastic_net_mse <- function(lst, X_tune, Y_tune, X_test, Y_test){
	P = length(lst)
	M = ncol(X_tune[[1]])
	lam_V = rep(0, P)
	test_res = list()
	test_beta = matrix(0, M, P)
	for(t in 1:P){
		ncv = length(lst[[t]]$lambda)
		tmp_mse = rep(0, ncv)
		for(k in 1:ncv){
			tmp_mse[k] = mean((Y_tune[[t]] - X_tune[[t]]%*%lst[[t]]$glmnet.fit$beta[,k])^2)
		}
		ss = which.min(tmp_mse)
		test_beta[,t] = lst[[t]]$glmnet.fit$beta[,ss]
		lam_V[t] = lst[[t]]$lambda[ss]
		predicted = X_test[[t]]%*%lst[[t]]$glmnet.fit$beta[,ss]
		test_res[[t]] = cbind(Y_test[[t]], predicted)
	}
	list(lam = lam_V, mse = test_res, est = test_beta)
}

multi_mse <- function(theta_est, X_test, Y_test){
	answer = list()
	P = ncol(theta_est)
	for(t in 1:P){
		predicted = X_test[[t]]%*%theta_est[,t]
		answer[[t]] = cbind(Y_test[[t]], predicted)
	}
	answer
}

avg_perm <- function(mse_lst){
	fd = length(mse_lst)
	P = length(mse_lst[[1]])
	rsq = mse = adj_mse = matrix(0, fd, P)
	for(f in 1:fd){
		for(t in 1:P){
			rsq[f,t] = (cor(mse_lst[[f]][[t]])[1,2])^2
			mse[f,t] = mean((mse_lst[[f]][[t]][,1]-mse_lst[[f]][[t]][,2])^2)
			adj_mse[f,t] = mse[f,t]/var(mse_lst[[f]][[t]][,1])
		}
	}
	cbind(apply(rsq, 2, mean), apply(mse, 2, mean), apply(adj_mse, 2, mean))

	#list(rsq = apply(rsq, 2, mean), mse = apply(mse, 2, mean), adj_mse = apply(adj_mse, 2, mean))
}

glasso <- function(X, Y, X1, Y1, XX, XY, Xnorm, lambda1, lambda2, theta, stepsize = 1e-4, maxiter = 10, eps = 1e-4){
	bgt = Sys.time()
	M = nrow(XY)
	P = length(X)
	NN = unlist(lapply(X, nrow))
	old_objV1 = 0
	for(t in 1:P){
		old_objV1 = old_objV1 + 1/2*mean((Y[[t]]-X[[t]]%*%theta[,t])^2)
	}
	IRdisplay::display_html(paste0("Training error: ", old_objV1, '\n'))
	old_objV2 = 0
	for(t in 1:P){
		old_objV2 = old_objV2 + 1/2*mean((Y1[[t]]-X1[[t]]%*%theta[,t])^2)
	}
	IRdisplay::display_html(paste0("Testing error: ", old_objV2, '\n'))
	beta_j_lasso = rep(0, P)
	tmp_XYj = 0
	if(!is.loaded("wrapper")){
		dyn.load("/ysm-gpfs/pi/zhao/from_louise/yh367/GTEX/codes/glasso.so")
	}
    
	for(i in 1:maxiter){
		bgt = Sys.time()
		res = .Call("wrapper", XX, XY, theta, M, P, beta_j_lasso, lambda1, lambda2, Xnorm)
		edt = Sys.time()
		#print(edt-bgt)
		new_objV1 = new_objV2 = 0
		for(t in 1:P){
			new_objV1 = new_objV1 + 1/2*mean((Y[[t]]-X[[t]]%*%theta[,t])^2)
            #IRdisplay::display_html(sum(is.na(X[[t]])))
            #IRdisplay::display_html(sum(is.na(Y[[t]])))
		}
		IRdisplay::display_html(paste0("Training error: ", new_objV1, '\n'))
		for(t in 1:P){
			new_objV2 = new_objV2 + 1/2*mean((Y1[[t]]-X1[[t]]%*%theta[,t])^2)
           # IRdisplay::display_html(sum(is.na(X[[t]])))
            #IRdisplay::display_html(sum(is.na(Y[[t]])))
		}
		IRdisplay::display_html(paste0("Testing error: ", new_objV2, '\n'))
		if(new_objV2 > old_objV2|new_objV1 > old_objV1){
			break
		}else{
			old_objV2 = new_objV2
		}
		if(abs(new_objV1-old_objV1) < eps){
			break
		}else{
			old_objV1 = new_objV1
		}
	}
	#edt = Sys.time()
	#print(edt-bgt)
	list(est = theta, tune_err = new_objV2)
}

glasso_no_early_stopping <- function(X, Y, XX, XY, Xnorm, lambda1, lambda2, theta, stepsize = 1e-4, maxiter = 250, eps = 1e-3){
	M = nrow(XY)
	P = length(X)
	NN = unlist(lapply(X, nrow))
	old_objV1 = 0
	for(t in 1:P){
		old_objV1 = old_objV1 + 1/2*mean((Y[[t]]-X[[t]]%*%theta[,t])^2)
	}
	IRdisplay::display_html(paste0("Training error: ", old_objV1, '\n'))
	beta_j_lasso = rep(0, P)
	tmp_XYj = 0
	if(!is.loaded("wrapper")){
		dyn.load("/ysm-gpfs/pi/zhao/from_louise/yh367/GTEX/codes/glasso.so")
	}
	for(i in 1:maxiter){
		res = .Call("wrapper", XX, XY, theta, M, P, beta_j_lasso, lambda1, lambda2, Xnorm)
		new_objV1 = 0
		for(t in 1:P){
			new_objV1 = new_objV1 + 1/2*mean((Y[[t]]-X[[t]]%*%theta[,t])^2)
		}
		IRdisplay::display_html(paste0("Training error: ", new_objV1, '\n'))
		if(abs(new_objV1-old_objV1) < eps|new_objV1 > old_objV1){
			break
		}else{
			old_objV1 = new_objV1
		}
	}
	list(est = theta, train_err = new_objV1)
}

extract_genotype <- function(gene_idx, chr_str, genotype_dir, gene_vec) {
    gene_dir <- paste0(genotype_dir, chr_str, gene_vec[gene_idx], "/", gene_vec[gene_idx])
    genotype_info <- read_plink(gene_dir)
    
    print(paste0("INFO: genotype matrix dimension:", dim(genotype_info$X)[1], " * ", dim(genotype_info$X)[2]))
    return(genotype_info)
}



In [3]:
# cv_config
fold = 5
cv_config = cv_helper(450, fold)

# create dirs
dir.create(paste0(output_dir, "chr", chr), showWarnings = FALSE)
dir.create(paste0(output_dir, "chr", chr, "/", gene_id), showWarnings = FALSE)
setwd(paste0(output_dir, "chr", chr, "/", gene_id))

# expr files 
cat("INFO: loading expression files...")
Y = list()
for(t in 1:P){
    Y[[t]] = read.table(paste0(gtex_dir, "adjusted_expr/chr", chr, "/", g, "/", Yt[t]), header=F)
}
ssize = unlist(lapply(Y, nrow))
T_num = length(Yt)

INFO: loading expression files...

In [ ]:
# cv_config
fold = 5
cv_config = cv_helper(450, fold)

# create dirs
dir.create(paste0(output_dir, "chr", chr), showWarnings = FALSE)
dir.create(paste0(output_dir, "chr", chr, "/", gene_id), showWarnings = FALSE)
setwd(paste0(output_dir, "chr", chr, "/", gene_id))

# expr files 
cat("INFO: loading expression files...")
Y = list()
for(t in 1:P){
    Y[[t]] = read.table(paste0(gtex_dir, "adjusted_expr/chr", chr, "/", g, "/", Yt[t]), header=F)
}
ssize = unlist(lapply(Y, nrow))
T_num = length(Yt)